## Connect Database

In [24]:
import csv
import mysql.connector
import pandas as pd
from configparser import ConfigParser
from DATA225utils import *

In [25]:
cursor.execute("DROP DATABASE IF EXISTS football")

In [26]:
cursor.execute("CREATE DATABASE football")

In [27]:
db_config = read_config('football.ini')

In [28]:
conn = make_connection(config_file = 'football.ini')
cursor = conn.cursor()

### Create teams database (schema)

In [29]:
# check data types of teamstats column
df = pd.read_csv("teams.csv")
dfstats = pd.read_csv("teamstats.csv")
dfstats.dtypes

gameID             int64
teamID             int64
season             int64
date              object
location          object
goals              int64
xGoals           float64
shots              int64
shotsOnTarget      int64
deep               int64
ppda             float64
fouls              int64
corners            int64
yellowCards      float64
redCards           int64
result            object
dtype: object

In [30]:
# Check if each dataframe has any NA value
df.isna().sum()

teamID    0
name      0
dtype: int64

In [31]:
dfstats.isna().sum()

gameID           0
teamID           0
season           0
date             0
location         0
goals            0
xGoals           0
shots            0
shotsOnTarget    0
deep             0
ppda             0
fouls            0
corners          0
yellowCards      1
redCards         0
result           0
dtype: int64

In [32]:
dfstats[dfstats['yellowCards'].isna()]

,gameID,teamID,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result
8280,4888,95,2014,2015-03-02 19:45:00,h,1,0.355629,8,3,2,8.1923,11,1,NaN,1,D


There is one row that has NA value in **`teamstats`** table, which is located in `yellowCards` column at 8280th row. We will change NA value to 0 later in ETL process.

### Create Database tables

In [33]:
# teams table

cursor.execute('DROP TABLE IF EXISTS teams')

sql = ( """
        CREATE TABLE teams
        (
            teamID       int,
            name         varchar(32),
            PRIMARY KEY(teamID)
        )
        """
      )

cursor.execute(sql);


# teamstats table

cursor.execute('DROP TABLE IF EXISTS teamstats')

sql = ("""
        CREATE TABLE teamstats
        (
            gameID             int,
            teamID             int,
            season             int,
            date              varchar(32),
            location          varchar(32),
            goals              int,
            xGoals           double,
            shots              int,
            shotsOnTarget      int,
            deep               int,
            ppda             double,
            fouls              int,
            corners            int,
            yellowCards        int,
            redCards           int,
            result            int,
            FOREIGN KEY (teamID) REFERENCES teams(teamID)
        )
        """
      )
cursor.execute(sql);

**`teams`** table has `teamID` as a `primary key` and **`teamstats`** table has `teamID` column as a `foreign key` that refers to teams table `teamID` Primary key.

A function to clean (transform) each row:¶
* yellowCards 'NA' ==> 0
* result 'W' ==> 2
* result 'D' ==> 1
* result 'L' ==> 0

Change `result` column to integer values.

In [34]:
def transform(row):
    if row[13] == 'NA':
        row[13] = 0

    _result = row[-1]

    if _result == 'W':
        row[-1] = 2
    elif _result == 'D':
        row[-1] = 1
    elif _result == 'L':
        row[-1] = 0
    else:
        try:
            row[13] = int(yellowCards)  # Convert other numeric values to int
        except ValueError:
            row[13] = 0

### SQL to Load two tables : teams, teamstats

In [35]:
sql_teams = (   """
              INSERT INTO teams
              VALUES (%s, %s)
              """
          )

sql_teamstats = (   """
               INSERT INTO teamstats
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
               """
           )

### Extract, transform, and load (ETL).

In [38]:
first = True
i = 0

with open('teams.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            if int(row[0]) in [71, 72]:
                cursor.execute(sql_teams, row)
            
        first = False
    
conn.commit()

In [39]:
first = True
i = 0

with open('teamstats.csv', newline='') as csv_file:
    data = csv.reader(csv_file, delimiter=',', quotechar='"')
    
    for row in data:
        if not first:
            transform(row)
            cursor.execute(sql_teamstats, row)
            
        first = False
        
        
conn.commit()

IntegrityError: 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`football`.`teamstats`, CONSTRAINT `teamstats_ibfk_1` FOREIGN KEY (`teamID`) REFERENCES `teams` (`teamID`))

### Display in a DataFrame

In [ ]:
# Frist 25 rows of teams table

sql = "SELECT * FROM teams LIMIT 25"

cursor.execute(sql)
rows  = cursor.fetchall()
count = cursor.rowcount

print(f'Fetched {count} rows.')
print()

for row in rows:
    print(row)

In [ ]:
# Frist 25 rows of teamstats table

sql = "SELECT * FROM teamstats LIMIT 25"

cursor.execute(sql)
rows  = cursor.fetchall()
count = cursor.rowcount

print(f'Fetched {count} rows.')
print()

for row in rows:
    print(row)

In [ ]:
# cursor.close()
# conn.close()

In [ ]:
x = 71
list